In [1]:
from datetime import timedelta
import pandas as pd
import requests
import os
TOKEN = os.getenv("AEMET_TOKEN")

In [2]:

MUNICIPALITY_PREDICTION = "https://opendata.aemet.es/opendata/api/prediccion/especifica/municipio/diaria/"

In [3]:
def run_query(base_url: str, endpoint: str):
    url = f"{base_url}/{endpoint}"
    headers = {"Authorization": f"Bearer {TOKEN}"}

    # Realizar la petición
    respuesta = requests.get(url, headers=headers)
    if respuesta.status_code == 200:
        datos = respuesta.json()
        # La API devuelve un enlace a los datos reales
        datos_url = datos.get("datos")
        if datos_url:
            datos_respuesta = requests.get(datos_url)
            return datos_respuesta.json()  # Devuelve los datos en formato JSON
        else:
            print("No se encontró la URL de los datos.")
    else:
        print(f"Error {respuesta.status_code}: {respuesta.text}")
        return None

# Obtener datos observados
vallehermoso = run_query(MUNICIPALITY_PREDICTION, "38050")

In [4]:
pred =vallehermoso[0].get("prediccion").get("dia")

In [48]:
def get_data_day(day, measure):
    day_of_pred = day.get("fecha")

    data = pd.DataFrame(day.get(measure))
    data["time"] = day_of_pred
    data.set_index("time", inplace=True)
    data.index = pd.to_datetime(data.index)

    return data

def get_data_day_with_hour(day, measure):
    data = get_data_day(day, measure)

    # Extraer 'value' y 'hora' de la columna 'dato'
    data["value"] = data["dato"].apply(lambda x: x.get("value"))
    data["hora"] = data["dato"].apply(
        lambda x: pd.Timedelta(
            hours=(x.get("hora") if x.get("hora") is not None and x.get("hora") != 24 else 23),
            minutes=(0 if x.get("hora") is not None and x.get("hora") != 24 else 59)
        )
    )

    data.index = pd.to_datetime(data.index) + data["hora"]

    data.drop(columns=["dato", "hora"], inplace=True)

    return data




for day in pred:
    # 1. probPrecipitacion
    df_precipitacion = get_data_day(day, "probPrecipitacion")
    # 2. cotaNieveProv
    df_nieve = get_data_day(day, "cotaNieveProv")
    # 4. estadoCielo
    # df_cielo = get_day_data(day, "estadoCielo")
    # 5. viento
    # df_viento = get_day_data(day, "viento")
    # 7. rachaMax
    df_rachamax = get_data_day(day, "rachaMax")
    # 8. temperatura
    df_temperatura = get_data_day_with_hour(day, "temperatura")
    # 9. sensTermica
    df_sesnstermica = get_data_day_with_hour(day, "sensTermica")
    # 10. humedadRelativa
    df_humedadrelativa = get_data_day_with_hour(day, "humedadRelativa")
    break

In [49]:
df_temperatura

,maxima,minima,value
2025-01-14 06:00:00,21,14,15
2025-01-14 12:00:00,21,14,19
2025-01-14 18:00:00,21,14,17
2025-01-14 23:59:00,21,14,15
